### Using the SageMaker TensorFlow Serving Container

[Documentation](https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-python-sdk/tensorflow_serving_container/tensorflow_serving_container.html)

Pueden utilizar esta vía para hacerle deploy a cualquier modelo independientemente de dónde lo entrenaron.

In [1]:
import os
import boto3
import sagemaker
import tarfile
import numpy as np
import cv2

from sagemaker.tensorflow import TensorFlow
from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.tensorflow.model import TensorFlowPredictor
from sagemaker.session import Session

from tensorflow.keras.datasets import mnist
from utils import to_tensor

sagemaker_role = sagemaker.get_execution_role()
session = Session()

2023-02-07 15:38:57.176661: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-07 15:39:10.541524: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Vamos a descargar el model que entrenamos en SageMaker para modificar la estructura de directorios con la que se creó y subirlo nuevamente a S3.

In [2]:
BUCKET = "sagemaker-us-east-1-720148074160"
TRAINED_MODEL_PATH = "sample-training-2023-02-01-14-32-59-634/output/model.tar.gz"

!rm -r ./model
!mkdir -p ./model

s3 = boto3.resource('s3')
s3.Bucket(BUCKET).download_file(TRAINED_MODEL_PATH, "model/model.tar.gz")

!tar -xvzf ./model/model.tar.gz --directory ./model/
!rm ./model/model.tar.gz
!mkdir -p ./model/0001
!mv ./model/assets ./model/0001/
!mv ./model/variables ./model/0001/
!mv ./model/*.pb ./model/0001/

!tar -czvf model.tar.gz ./model/
!rm -r ./model

model_data = session.upload_data(
    path="model.tar.gz", 
    key_prefix="model"
)
print(f"Model uploaded to: {model_data}")

rm: cannot remove ‘./model’: No such file or directory
variables/
variables/variables.index
variables/variables.data-00000-of-00001
assets/
saved_model.pb
./model/
./model/0001/
./model/0001/variables/
./model/0001/variables/variables.index
./model/0001/variables/variables.data-00000-of-00001
./model/0001/saved_model.pb
./model/0001/assets/
Model uploaded to: s3://sagemaker-us-east-1-720148074160/model/model.tar.gz


Deploy the model

In [3]:
model = TensorFlowModel(
    model_data=model_data, 
    role=sagemaker_role, 
    framework_version="2.4"
)

predictor = model.deploy(initial_instance_count=1, instance_type="ml.c5.xlarge")

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


----!

In [4]:
for image_path in os.listdir("./images"):
    if image_path.endswith(".png"):

        # Pre
        image = to_tensor(os.path.join("./images", image_path))
        
        # Prediction
        result = predictor.predict(image)
        
        # Post
        prediction = np.argmax(result["predictions"])
        print(f"{image_path} ---> {prediction}")
        

7.png ---> 7
1.png ---> 1
2.png ---> 2


In [5]:
predictor.delete_endpoint()